In [11]:
import mysql.connector
import pandas as pd
import os
import numpy as np
import re
import openai
import sys
import tiktoken
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

def sql_db_recall(name):
    """ mysql을 통해 SQL DB에서 dataframe 형태로 변환하여 필요한 정보들을 불러오는 함수/
        한의원의 이름을 입력하면 해당 한의원의 데이터들을 SQL DB에서 불러온다. 
    """
    # sql db에 접속
    cnx = mysql.connector.connect(user='fastcampus', 
                              password='####', 
                              host='####', 
                              database = 'members')
    # members table 불러오기
    members_query = "SELECT * FROM members"
    members = pd.read_sql(members_query, cnx)
    # 월 매출 table 불러오기
    monthly_revenue_query = "SELECT * FROM monthly_revenue"
    monthly_revenue_df = pd.read_sql(monthly_revenue_query, cnx)
    # 월 예약 table 불러오기
    monthly_appointment_query = "SELECT * FROM monthly_appointment"
    monthly_appointment_df = pd.read_sql(monthly_appointment_query, cnx)
    # 월 비급여 상담 table 불러오기
    monthly_nonbenefit_cnsl_query = "SELECT * FROM monthly_nonbenefit_cnsl"
    monthly_nonbenefit_cnsl_df = pd.read_sql(monthly_nonbenefit_cnsl_query, cnx)
    # 월 비급여 재결제 상담 table 불러오기
    monthly_nonbenefit_repay_cnsl_query = "SELECT * FROM monthly_nonbenefit_repay_cnsl"
    monthly_nonbenefit_repay_cnsl_df = pd.read_sql(monthly_nonbenefit_repay_cnsl_query, cnx)
    # 월 환자 table 불러오기
    monthly_patients_query = "SELECT * FROM monthly_patients"
    monthly_patients_df = pd.read_sql(monthly_patients_query, cnx)
    # sql db 접속 종료
    cnx.close()

    # members table에서 병원 이름과 고유번호 쌍 만들어 저장
    name_mapping = members[['id','hospital_nm']].to_dict('records')
    global new_name_mapping
    for i in range(len(name_mapping)):
        new_name_mapping.append(name_mapping[i].values())
    new_name_mapping = dict(new_name_mapping)

    # 불러온 월별 table들을 1차 전처리
    monthly_revenue_df = table_preprocessing(monthly_revenue_df)
    monthly_appointment_df = table_preprocessing(monthly_appointment_df)
    monthly_nonbenefit_cnsl_df = table_preprocessing(monthly_nonbenefit_cnsl_df)
    monthly_nonbenefit_repay_cnsl_df = table_preprocessing(monthly_nonbenefit_repay_cnsl_df)
    monthly_patients_df = table_preprocessing(monthly_patients_df)

    # 각 table별 필요한 컬럼들 리스트
    pay_col = ['biz_dt', 'total_pay', 'nonbenefit_total_pay', 'carinsur_pay', 'nhis_pay', 'co_pay', 'room_pay','etc_pay']
    app_col = ['biz_dt','appointment_cnt','fulfillment_cnt','cancel_cnt','noshow_cnt','reception_cnt']
    non_cnsl_col = ['biz_dt','nonbenefit_cnsl_cnt','nonbenefit_cnsl_succ_cnt','nonbenefit_cnsl_fail_cnt']
    non_cnsl_repay_col=['biz_dt','nonbenefit_repay_cnsl_cnt','nonbenefit_repay_cnsl_succ_cnt','nonbenefit_repay_cnsl_fail_cnt']
    patient_col=['biz_dt','visit_cnt','fstvisit_cnt','revisit_cnt','refstvisit_cnt','carinsur_cnt','senior_cnt','room_cnt']

    # 각 데이터프레임에서 타겟 한의원 필터링 및 필요한 컬럼들 선택
    filtered_revenue_df = monthly_revenue_df[monthly_revenue_df['fk_id'] == name][pay_col]
    filtered_appointment_df = monthly_appointment_df[monthly_appointment_df['fk_id'] == name][app_col]
    filtered_nonbenefit_cnsl_df = monthly_nonbenefit_cnsl_df[monthly_nonbenefit_cnsl_df['fk_id'] == name][non_cnsl_col]
    filtered_nonbenefit_repay_cnsl_df = monthly_nonbenefit_repay_cnsl_df[monthly_nonbenefit_repay_cnsl_df['fk_id'] == name][non_cnsl_repay_col]
    filtered_patients_df = monthly_patients_df[monthly_patients_df['fk_id'] == name][patient_col]

    # 데이터프레임 병합
    merged_df = filtered_revenue_df.merge(filtered_appointment_df, on='biz_dt', how='inner') \
    .merge(filtered_nonbenefit_cnsl_df, on='biz_dt', how='inner') \
    .merge(filtered_nonbenefit_repay_cnsl_df, on='biz_dt', how='inner') \
    .merge(filtered_patients_df, on='biz_dt', how='inner')

    return merged_df

def table_preprocessing(df):
    """
    sql에서 불러온 데이터를 1차적으로 전처리하는 함수
    """
    global new_name_mapping # 전역변수 호출
    # 1. 병원 고유번호를 이름으로 대치
    df['fk_id'] = df['fk_id'].replace(new_name_mapping)
    # 2. 병원이름순으로 정렬후, 불필요한 컬럼 제거
    df = df.sort_values(by='fk_id').reset_index().drop(columns=['biz_cnt','index','data_source'])
    # 3. 'biz_dt' column을 to_datetime 으로 형변환
    df['biz_dt'] = pd.to_datetime(df['biz_dt'],errors='coerce')
    # 4. datetime 데이터 오류로 2018~2023의 데이터만 추출
    df = df[(df['biz_dt'].dt.year >=2018) & (df['biz_dt'].dt.year <=2023)].sort_values(by=['fk_id','biz_dt'])

    return df

def preprocessing(df):
    """
    데이터프레임 전처리 함수. null값 제거, row수 정리, 한글 columns, 추가 columns 등의 기능을 한다.
    """

    # 한 column에 대해 전체가 null값일 때 제거
    merged_df =df.dropna(axis=1, how='all')
    # 한 row에 대해서 모두 null값이거나 0이면 그 row 제거 (날짜는 항상 채워져있으므로 제외시킴)
    df_col_tolist = merged_df.columns.tolist()
    df_col_tolist.pop(0)
    merged_df = merged_df.dropna(axis =0, subset=df_col_tolist, how='all')
    merged_df = merged_df.loc[:, (merged_df != 0).any(axis=0)]

    # 데이터프레임 한의원 처음3개월 + 최근6개월 데이터
    if len(merged_df) > 9:
        df = pd.concat([merged_df.head(3), merged_df.tail(6)], ignore_index=True)

    # column명 한글 수정
    df.rename(columns={
        'biz_dt': '월별 날짜',
        'total_pay': '총 매출',
        'nonbenefit_total_pay': '비급여 총매출', 
        'carinsur_pay': '자보매출',
        'nhis_pay': '공단청구금',
        'co_pay': '본인부담금',
        'room_pay': '입원실 매출',
        'etc_pay': '기타매출',
        'visit_cnt': '환자 수',
        'fstvisit_cnt': '신규 환자수',
        'revisit_cnt': '재진 환자수',
        'refstvisit_cnt': '재초진 환자수',
        'room_cnt': '입원실 환자수',
        'carinsur_cnt': '자보초진 환자수',
        'senior_cnt': '65세이상 환자수',
        'appointment_cnt': '예약 환자수',
        'reception_cnt': '데스크 예약수',
        'fulfillment_cnt': '정상이행 수',
        'cancel_cnt': '취소 수',
        'noshow_cnt': '노쇼 수',
        'nonbenefit_cnsl_cnt': '비급여 초진상담수',
        'nonbenefit_cnsl_succ_cnt': '비급여 초진치료전환 수',
        'nonbenefit_cnsl_fail_cnt': '비급여 초진전환실패 수',
        'nonbenefit_repay_cnsl_cnt': '비급여 재결제상담수',
        'nonbenefit_repay_cnsl_succ_cnt': '비급여 재결제 치료전환 수',
        'nonbenefit_repay_cnsl_fail_cnt': '비급여 재결제 전환실패 수',
          }, inplace=True)

    # 비율 column추가
    if "비급여 총매출" in df:
        df['비급여 총매출/총 매출 (%)'] = df["비급여 총매출"] / df["총 매출"] * 100
    if "신규 환자수" in df.columns:
        df['신규 환자수/환자수 (%)'] = df["신규 환자수"] / df["환자 수"] * 100
    if "재진 환자수" in df:
        df['재진 환자수/환자 수 (%)'] = df["재진 환자수"] / df["환자 수"] * 100

    return df

def col_matching(df):
    """
    전처리가 완료된 데이터 프레임의 column 값들에 맞는 설명을 불러오는 함수. 
    """
    # 주어진 dataframe의 column 값들을 list로 반환
    existing_cols = df.columns.to_list()
    # 모든 column 값들에 대한 설명 dict
    col_dict = {
        '월별 날짜': '한의원의 월별 날짜',
        '총 매출': "It's total sales of medical clinic for a month. It's the sum of '비급여 총매출', '자보매출', '공단청구금', '본인부담금', '입원실 매출', '기타매출'.",
        '비급여 총매출': 'This sales are paid by patient 100%. It is important for medical clinics to induce sales of these items to increase total sales.',
        '자보매출': 'These sales are sales that auto insurance companies provide medical expenses for patients who come to medical clinics due to traffic accidents.',
        '공단청구금': 'These sales are paid by the Health Care Corporation, etc., which patients who visited the medical clinic joined.',
        '본인부담금': "These are the remaining expenses after paying a portion of the patient's medical expenses from the corporation or insurance. It's the sales that the patient pays directly to the hospital.",
        '입원실 매출': 'These sales are paid by inpatients during their hospitalization period.',
        '기타매출': 'These are sales that occurred outside of medical treatment.',
        '환자 수': 'This indicates the total number of patients who visited the hospital for a month.',
        '신규 환자수': 'This indicates the number of patients who visited the hospital for the first time.',
        '재진 환자수': "This indicates the number of patients for a month, who visitied the hospital before and still haven't got better with their illness.",
        '자보초진 환자수': "This indicates the number of patients who visitied the hospital for the first time, because of car accidents and are deeply related to '자보매출'.",
        '65세이상 환자수': 'This indicates the number of patients above 65 years of age, among the patients who visited the hospital.',
        '입원실 환자수': 'This indicates the number of patients of those in their hospitalization period.',
        '재초진 환자수': "This indicates the number of patients who visited the hospital with new issue of their body. They've visited to the hospital before and had got better with their original illness.",
        '예약 환자수' : 'Number of patients who made appointment for next visit.',
        '데스크 예약수': 'Number of patients who made appointment for next visit at the reception of the hospital. Usaually those who came to medical clinic for treatment, and not finished with the whole treatment process.',
        '정상이행 수': 'Number of patients who fullfilled the appointment that the patient had made.',
        '취소 수': 'Number of patients who canceled their appointment.',
        '노쇼 수': 'Number of patients who did not show on their appointment without any notice.',
        '비급여 초진상담수': "Number of consultations of first visited patients, about treatments that patient should pay 100%. These treatments are highly related to '비급여 총매출'.",
        '비급여 초진치료전환 수': "Number of consultations of first visited patients about treatments that patient should pay 100%, that successfully converted to sales.",
        '비급여 초진전환실패 수': "Number of consultations of first visited patients about treatments that patient should pay 100%, that failed to be converted to sales.",
        '비급여 재결제상담수': 'Number of consultations of revisted patients about treatments that patient should pay 100%.',
        '비급여 재결제 치료전환 수': 'Number of consultations of revisted patients about treatments that patient should pay 100%, that successfully converted to sales.',
        '비급여 재결제 전환실패 수': 'Number of consultations of revisted patients about treatments that patient should pay 100%, that failed to be converted to sales.',
        '신규 환자수/환자수 (%)': "This is a result of 'fstvisit_cnt' divided by 'visit_cnt'.",
        '비급여 총매출/총 매출 (%)': "This is a result of 'nonbenefit_total_pay' divided by 'total_pay'.",
        '재진 환자수/환자 수 (%)': "This is a result of 'revisit_cnt' divided by 'visit_cnt'."
    }

    # dataframe에 해당하는 column 값들의 설명들만 선택
    col_prompt=""
    for i in existing_cols:
        if i in col_dict.keys():
            col_prompt += (f"""-'{i}': {col_dict.get(i)}\n    """)
            
    return col_prompt

def gpt_answer(df):
    """
    GPT 4.0에게 프롬프트를 입력하여 결과를 불러오는 함수. Dataframe을 input한다.
    """

    openai.api_key="@@####"

    answer=[]

    high_similarity_questions_2 =[
    '초진 이탈율, 삼진율 데이터가 정상적으로 입력되고 있을까요? 지표 개선을 위한 정책 고민해주세요. 아래는 1월 지표입니다',
    '비급여 매출 2월 하락요인과 개선 전략은?',
    '원장님 지난 6월 직원 교육을 통해 환자 커뮤니케이션 개선 말씀주셨습니다. 7월 결과가 좋았고 8월에 주춤했지만 9월 다시 반등했습니다. 어떤 요인의 결과로 보실까요?',
    '신규환자 유입 비중에서 ‘인터넷’ 이 2% 로 가장 낮습니다. 플레이스 검색 순위에서 상위권 내 포지셔닝 하기 위한 방안 2가지 말씀 부탁드립니다.',
    '주력 비급여 중 하나인 소아정신과의 매출 하락의 원인을 어떻게 평가하고 계신가요?'
    ]
    system_prompt ="""
                    You are a helpful assistant.
                   """

    user_prompt = f"""

        You are an expert in creating feedback questions from management indicator.
        "타겟 데이터테이블"은 특정 한의원에 대한 데이터이며, 너에게 입력될 string형태로 표현된 dataframe 데이터야.
        "타겟 데이터테이블"의 컬럼들을 복합적으로 분석해서 큰 특이점 또는 추세를 다양하게 7가지 알려줘.
        그리고 이 특이점과 추세들을 기준으로 한의원 경영을 위한 창의적인 피드백 질문들을 7개 생성해줘.
        "타겟 데이터테이블"에 없는 날짜나 내용은 절대 언급하지마.
        
        - 가장 마지막 달과 최근 3개월의 기간을 중요하게 관찰하고 비교군으로 적극 활용해라.
        - 오직 '타겟 데이터테이블'에 있는 내용으로만 분석해야한다.
        - "Columns explanation"은 '타겟 데이터테이블'의 각각 컬럼들에 대한 설명이다.
        - "참고"에서 피드백 질문의 말투를 참고하고, "타겟 데이터테이블"에 없는 내용은 절대 언급하지마.
        - "타겟 데이터테이블"의 아래로 갈수록 최근 날짜의 데이터이다.
        - "가장"이라는 극단적인 단어 표현 대신, "비교적 크게" 혹은 "비교적 많이"처럼 순화적인 표현을 사용해줘.
        - 특이점과 피드백 질문을 생성할 때, 구체적인 날짜나 기간도 언급해줘.
        - "타겟 데이터테이블"에 없는 컬럼이나 날짜는 절대 언급하지마.
        - 사실적인 질문보다는 추론적인 피드백 질문을 해줘.
        - 피드백질문을 창의적으로 생성해줘.
        - 특이점은 타겟 데이터에 기반한 정확성이 중요해.
        - 특이점 하나, 그에 대한 피드백 질문 하나 이런 방식으로 생성해줘.
        
        Columns explanation:
        {col_matching(df)}
        참고 :
        {high_similarity_questions_2}
        타겟 데이터테이블:
        {df.to_string(index=False)}
        """

    completion = openai.ChatCompletion.create(
        model = "gpt-4",
        messages=[{"role": "system", "content": system_prompt},
                   {"role": "user", "content": user_prompt}]
    )

    for choice in completion.choices:
        print(choice['message']['content'])
        answer.append(choice['message']['content'].strip())

    return answer

def num_tokens_from_messages(df, messages, model="gpt-4"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("\n\nWarning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(df, messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("\n\nWarning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(df, messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    
    for i in messages:
        answer = i

    high_similarity_questions_2 =[
        '초진 이탈율, 삼진율 데이터가 정상적으로 입력되고 있을까요? 지표 개선을 위한 정책 고민해주세요. 아래는 1월 지표입니다',
        '비급여 매출 2월 하락요인과 개선 전략은?',
        '원장님 지난 6월 직원 교육을 통해 환자 커뮤니케이션 개선 말씀주셨습니다. 7월 결과가 좋았고 8월에 주춤했지만 9월 다시 반등했습니다. 어떤 요인의 결과로 보실까요?',
        '신규환자 유입 비중에서 ‘인터넷’ 이 2% 로 가장 낮습니다. 플레이스 검색 순위에서 상위권 내 포지셔닝 하기 위한 방안 2가지 말씀 부탁드립니다.',
        '주력 비급여 중 하나인 소아정신과의 매출 하락의 원인을 어떻게 평가하고 계신가요?'
        ]

    system_prompt ="""
                    You are a helpful assistant.
                    """

    user_prompt = f"""

        You are an expert in creating feedback questions from management indicator.
        "타겟 데이터테이블"은 특정 한의원에 대한 데이터이며, 너에게 입력될 string형태로 표현된 dataframe 데이터야.
        "타겟 데이터테이블"의 컬럼들을 복합적으로 분석해서 큰 특이점 또는 추세를 다양하게 7가지 알려줘.
        그리고 이 특이점과 추세들을 기준으로 한의원 경영을 위한 창의적인 피드백 질문들을 7개 생성해줘.
        "타겟 데이터테이블"에 없는 날짜나 내용은 절대 언급하지마.
        
        - 가장 마지막 달과 최근 3개월의 기간을 중요하게 관찰하고 비교군으로 적극 활용해라.
        - 오직 '타겟 데이터테이블'에 있는 내용으로만 분석해야한다.
        - "Columns explanation"은 '타겟 데이터테이블'의 각각 컬럼들에 대한 설명이다.
        - "참고"에서 피드백 질문의 말투를 참고하고, "타겟 데이터테이블"에 없는 내용은 절대 언급하지마.
        - "타겟 데이터테이블"의 아래로 갈수록 최근 날짜의 데이터이다.
        - "가장"이라는 극단적인 단어 표현 대신, "비교적 크게" 혹은 "비교적 많이"처럼 순화적인 표현을 사용해줘.
        - 특이점과 피드백 질문을 생성할 때, 구체적인 날짜나 기간도 언급해줘.
        - "타겟 데이터테이블"에 없는 컬럼이나 날짜는 절대 언급하지마.
        - 사실적인 질문보다는 추론적인 피드백 질문을 해줘.
        - 피드백질문을 창의적으로 생성해줘.
        - 특이점은 타겟 데이터에 기반한 정확성이 중요해.
        - 특이점 하나, 그에 대한 피드백 질문 하나 이런 방식으로 생성해줘.
        
        Columns explanation:
        {col_matching(df)}
        참고 :
        {high_similarity_questions_2}
        타겟 데이터테이블:
        {df.to_string(index=False)}
        """
    prompt_q=[{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    
    prompt_a=[{"role": "assistant", "content": answer}]

    num_tokens_q = 0
    for message in prompt_q:
        num_tokens_q += tokens_per_message
        for key, value in message.items():
            num_tokens_q += len(encoding.encode(value))
            if key == "name":
                num_tokens_q += tokens_per_name
    num_tokens_q += 3  # every reply is primed with <|start|>assistant<|message|>
    
    num_tokens_a = 0
    for message in prompt_a:
        num_tokens_a += tokens_per_message
        for key, value in message.items():
            num_tokens_a += len(encoding.encode(value))
            if key == "name":
                num_tokens_a += tokens_per_name
    num_tokens_a += 3  # every reply is primed with <|start|>assistant<|message|>    

    print(f"""
    Q_tokens_cnt: {num_tokens_q}
    A_tokens_cnt: {num_tokens_a}
    Total_tokens_cnt: {num_tokens_a + num_tokens_q}
    Cost: {round((num_tokens_q/1000*0.03) + (num_tokens_a/1000*0.06), 2)}$""")

# 한의원 이름 입력 받기
while True:
    # global 변수 new_name_mapping 설정
    new_name_mapping=[]

    clinic_name = input("한의원 명 : ")
    # 종료 명령어 'exit'
    if clinic_name == 'exit':
        sys.exit()

    # sql에서 해당 한의원 dataframe 생성        
    df = sql_db_recall(clinic_name)
    
    # 이름이 틀릴경우 다시 반복
    if not df.empty:
        break
print(f"""한의원 명 : {clinic_name}\n""")

# dataframe 전처리
preprocessed_df = preprocessing(df)

# 비교를 위한 dataframe 출력
display(preprocessed_df)

# gpt 4.0으로부터 답변 구하기
ans = gpt_answer(preprocessed_df)

# token 갯수 구하기
num_tokens_from_messages(preprocessed_df,ans)

특이점 1: 최근 3개월 동안(2020년 11월, 12월, 2021년 1월) 비급여 총매출/총 매출(%)이 전체 기간 중 가장 낮게 나타났습니다. 
피드백 질문 1: 비급여 총매출 비율이 최근 3개월 동안 낮아진 이유는 무엇이라고 생각하시나요? 새로운 방식이나 비급여 서비스를 도입할 계획이 있으신가요?

특이점 2: 2021년 2월에 환자 수가 가장 높게 나타났습니다.
피드백 질문 2: 2021년 2월 환자 수가 급증한 원인은 무엇이라고 생각하시나요? 그 원인을 유지하고 지속적으로 환자 수를 높이기 위한 계획이 있으신가요?

특이점 3: 최근 3개월 동안(2020년 12월, 2021년 1월, 2월) 자보초진 환자수가 상대적으로 낮게 나타났습니다.
피드백 질문 3:자동차 사고 환자에 대한 적극적인 관리와 의료보험 지원을 제공하여 자보초진 환자수를 늘리기 위한 계획이 있으신가요?

특이점 4: 2019년 2월 대비 2020년 11월에 신규 환자수/환자수(%)가 상대적으로 높게 나타났습니다.
피드백 질문 4: 2019년 2월에 비해 2020년 11월에 신규 환자수 비율이 높아진 이유가 무엇이라고 생각하시나요? 지난 달 신규 행사, 광고, 또는 마케팅 전략 중 어떤 것들이 효과가 있었나요?

특이점 5: 2020년 9월부터 2021년 2월까지의 기간 동안 재진 환자수/환자 수(%)가 전체 기간 중 가장 높게 나타났습니다.
피드백 질문 5: 지난 6개월 동안 재진 환자 비율이 높아진 이유는 무엇이라고 생각하시나요? 재진 환자들의 만족도를 높이거나 그들이 지속적으로 치료를 받을 수 있도록 할 계획이 있으신가요?

특이점 6: 2019년 3월과 2020년 11월에 자보매출이 상대적으로 높게 나타났습니다.
피드백 질문 6:한의원에 처음 방문하는 교통사고 환자에게 어떤 서비스를 제공하여 이들이 다른 한의원 대신 본 한의원을 선택하도록 할 계획이 있으신가요?

특이점 7: 2019년 1월과 2021년 2월에 본인부담금이 상대적으로 높게 나타났습니다.
피드백 질문 7: 순수 치료